In [2]:
import tensorflow as tf
import numpy as np
import os 
import cv2
import random
from tqdm import tqdm
from tensorflow.keras.layers import LSTM, Bidirectional, Conv2D, Dense, Flatten, MaxPooling2D, TimeDistributed, Reshape
from tensorflow.keras.models import Sequential
import time
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from random import shuffle
from sklearn.model_selection import KFold
from tensorflow.keras import optimizers

In [5]:
LR = 0.0001
BATCH_SIZE = 64
MOMENTUM = 0.9
IMG_SIZE = 224
EPOCHS = 5

TRAIN_DIR = './Peliculas/Data/train/'
TEST_DIR = './Peliculas/Data/valid/'

In [17]:
TRAIN_DATA = []
for folder in os.listdir(TRAIN_DIR):
    for files in tqdm(os.listdir(os.path.join(TRAIN_DIR, folder))):
        path = os.path.join(os.path.join(TRAIN_DIR, folder), files)
        image = cv2.resize(cv2.imread(path), (IMG_SIZE, IMG_SIZE))
        if folder == 'fight':
            TRAIN_DATA.append([np.array(image), [1, 0]])
        elif folder == 'nofight':
            TRAIN_DATA.append([np.array(image), [0, 1]])

shuffle(TRAIN_DATA)
np.save('movie_training_data.npy', TRAIN_DATA)

100%|██████████| 4410/4410 [00:18<00:00, 243.31it/s]


In [18]:
TEST_DATA = []
for folder in os.listdir(TEST_DIR):
    for files in tqdm(os.listdir(os.path.join(TEST_DIR, folder))):
        path = os.path.join(os.path.join(TEST_DIR, folder), files)
        image = cv2.resize(cv2.imread(path), (IMG_SIZE, IMG_SIZE))
        if folder == 'fight':
            TEST_DATA.append([np.array(image), [1, 0]])
        elif folder == 'nofight':
            TEST_DATA.append([np.array(image), [0, 1]])

shuffle(TEST_DATA)
np.save('movie_testing_data.npy', TEST_DATA)

100%|██████████| 539/539 [00:01<00:00, 274.18it/s]


In [3]:
def create_dataset():
    dataset = []
    limit = 0


    for file in tqdm(os.listdir('./Peliculas/Dataframes/')):
        path = os.path.join('./Peliculas/Dataframes/', file)
        image = cv2.resize(cv2.imread(path), (IMG_SIZE, IMG_SIZE))
        if limit < 4791:
            dataset.append([image, np.array([1, 0])])
        elif limit >= 4791:
            dataset.append([image, np.array([0, 1])])
        
        limit += 1 
            
    
    shuffle(dataset)
    np.save('movie_data.npy', dataset)
    return dataset

In [4]:
data = create_dataset()

100%|██████████| 9841/9841 [01:10<00:00, 140.10it/s]
C:\Users\User\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [3]:
data = np.load('movie_data.npy', allow_pickle=True)

In [23]:
data[0][1]

array([0, 1])

In [15]:
DATA_SET = [] 
#for folders in os.listdir('./Peliculas/Data/'+str(folders)): 
folder_path = './Peliculas/Videos/' 
for files in tqdm(os.listdir(folder_path)):     
    print(files)     
    path = os.path.join(folder_path, files)     
    print(path)     
    for a in os.listdir(path):  
        paths= os.path.join(path,a)         
        image = cv2.resize(cv2.imread(paths), (IMG_SIZE, IMG_SIZE))

        if files == 'fight':
            DATA_SET.append([np.array(image), [1, 0]])         
        elif files == 'nofight':             
            DATA_SET.append([np.array(image), [0, 1]])  
            
shuffle(DATA_SET) 
np.save('movie_data_1.npy', DATA_SET)

  0%|          | 0/2 [00:00<?, ?it/s]fights
./Peliculas/Videos/fights



error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-6lylwdcz\opencv\modules\imgproc\src\resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [11]:
base_model = tf.keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
)

base_model.trainable = False

add_model = Sequential()
add_model.add(base_model)
add_model.add(GlobalAveragePooling2D())
add_model.add(Dense(2, activation='softmax'))

model = add_model
model.compile(loss='categorical_crossentropy', optimizer = optimizers.SGD(lr=LR, momentum = 0.9), metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 21,806,882
Trainable params: 4,098
Non-trainable params: 21,802,784
_________________________________________________________________


In [24]:
kf = KFold(10)

X = np.array([i[0] for i in data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y = np.array([i[1] for i in data])
X = X.astype('float32')/255
fold = 0

for train, test in kf.split(data):
    fold += 1
    x_train = X[train]
    y_train = Y[train]
    x_test = X[test]
    y_test = Y[test]

    model.fit(x_train, y_train, validation_data=(x_test, y_test), verbose = 1, epochs= 5, batch_size=64)
    Model_name = 'movie-model-fold' + str(fold) + '.h5'
    #weights_name = 'movie-weight-fold' + str(fold) + '.h5'
    model.save(Model_name, overwrite = True, include_optimizer = True)
    #model.save_weights(weights_name, overwrite=True)

Epoch 1/5
139/139 [==============================] - 29s 198ms/step - loss: 0.6715 - accuracy: 0.6740 - val_loss: 0.5598 - val_accuracy: 0.7574
Epoch 2/5
139/139 [==============================] - 26s 190ms/step - loss: 0.5002 - accuracy: 0.7894 - val_loss: 0.4639 - val_accuracy: 0.8152
Epoch 3/5
139/139 [==============================] - 27s 193ms/step - loss: 0.4345 - accuracy: 0.8266 - val_loss: 0.4115 - val_accuracy: 0.8396
Epoch 4/5
139/139 [==============================] - 27s 192ms/step - loss: 0.3938 - accuracy: 0.8471 - val_loss: 0.3790 - val_accuracy: 0.8508
Epoch 5/5
139/139 [==============================] - 27s 194ms/step - loss: 0.3651 - accuracy: 0.8577 - val_loss: 0.3505 - val_accuracy: 0.8589
Epoch 1/5
139/139 [==============================] - 27s 193ms/step - loss: 0.3384 - accuracy: 0.8688 - val_loss: 0.3617 - val_accuracy: 0.8587
Epoch 2/5
139/139 [==============================] - 28s 199ms/step - loss: 0.3202 - accuracy: 0.8728 - val_loss: 0.3437 - val_accuracy:

In [25]:
model.evaluate(x = X, y =Y, batch_size= 5, verbose = 1)

1969/1969 [==============================] - 71s 36ms/step - loss: 0.1444 - accuracy: 0.9525


[0.1444113701581955, 0.9525454640388489]

In [6]:
X = np.array([i[0] for i in data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y = np.array([i[1] for i in data])

In [7]:
X = X.astype('float32')/255

In [8]:
model_tf = load_model('./movie-model-fold10.h5')
model_tf.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 21,806,882
Trainable params: 4,098
Non-trainable params: 21,802,784
_________________________________________________________________


In [9]:
model_tf.evaluate(x = X, y =Y, batch_size= 5, verbose = 1)

1969/1969 [==============================] - 65s 32ms/step - loss: 0.1444 - accuracy: 0.9525


[0.14441141486167908, 0.9525454640388489]